In [63]:
#IMPORTS

import pandas as pd
import numpy as np
import requests
import os
import matplotlib.pyplot as plt
from tabulate import tabulate 

In [64]:
#HELPER FUNCTIONS

def get_ranked(df, sort_column, sort_flag, num_rows):
    '''Sort dataframe according to parameters and display specified number of rows as result'''
    df = df.sort_values(sort_column, ascending=sort_flag).head(num_rows)
    df.reset_index(inplace=True, drop=True)
    return df

def print_table(table, keys='keys'):
    '''Tabulate dataframe with nice formatting'''
    print(tabulate(table, headers=keys, tablefmt='fancy_outline'))

def plot_stats(df, flag):
    '''Plot the statistics dataframe with scale in either days or years'''
    if flag == 'years':
        df = df.div(365.25, axis=0)

    plt.figure()
    ax = df.transpose().plot.bar(xlabel='Statistic', ylabel='Value (in {})'.format(flag), legend=None)
    ax.set_xticklabels(ax.get_xticklabels(), rotation = 0)
    plt.axhline(y=df['mean'][0], color='r')
    plt.legend(['Mean'])
    plt.savefig('plots/Statistics_{}.png'.format(flag))
    plt.close()

In [65]:
#PRELIMINARY PROCESSING

#input .csv file
input_file_name = 'U.S. Presidents Birth and Death Information - Sheet1.csv'
input_file_url = 'https://raw.githubusercontent.com/senrabc/a_problem_with_presidents/main/' + input_file_name

#output plot directory
output_plot = 'plots'

#check if file is available locally, else download it
if(not os.path.isfile(input_file_name)):
    res = requests.get(input_file_url)
    open(input_file_name, 'wb').write(res.content)

#check if plots/ directory exists, if not, create it
if(not os.path.isdir(output_plot)):
    os.makedirs((output_plot))

In [66]:
#WORKING WITH THE DATAFRAME

#create dataframe from input csv
df = pd.read_csv(input_file_name)

#remove 'References' row in the dataframe
df.drop(df.tail(1).index, inplace=True)

#Convert to convenient Datetime objects
df['cleanedDOB'] = pd.to_datetime(df['BIRTH DATE']) 
df['cleanedDOD'] = pd.to_datetime(df['DEATH DATE'])


In [67]:
#calculate year_of_birth
df['year_of_birth'] = df['cleanedDOB'].dt.year
#current day normalized to midnight 00:00:00
today = pd.Timestamp.now().normalize()
#calculate days lived
df['lived_days'] = (df['cleanedDOD'] - df['cleanedDOB']).where(df['cleanedDOD'].notnull(), other=today - df['cleanedDOB']).dt.days
#calculate years lived
df['lived_years'] = (df['lived_days'] / 365.25).astype(int)
#calculate months lived
df['months_lived'] = (df['lived_days'] / 12.0).astype(int)

In [68]:
#top 10 longest lived presidents
longest_lived = get_ranked(df, 'lived_days', False, 10)

#top 10 shortest lived presidents
shortest_lived = get_ranked(df, 'lived_days', True, 10)

#print top 10 presidents
columns_to_display = ['PRESIDENT', 'BIRTH DATE', 'DEATH DATE', 'lived_days', 'lived_years']

print('Top 10 longest lived presidents')
print_table(longest_lived[columns_to_display])

print('Top 10 shortest lived presidents')
print_table(shortest_lived[columns_to_display])

Top 10 longest lived presidents
╒════╤══════════════════╤═══════════════╤═══════════════╤══════════════╤═══════════════╕
│    │ PRESIDENT        │ BIRTH DATE    │ DEATH DATE    │   lived_days │   lived_years │
╞════╪══════════════════╪═══════════════╪═══════════════╪══════════════╪═══════════════╡
│  0 │ Jimmy Carter     │ Oct 1, 1924   │ nan           │        35870 │            98 │
│  1 │ George Bush      │ June 12, 1924 │ Nov 30, 2018  │        34504 │            94 │
│  2 │ Gerald Ford      │ July 14, 1913 │ Dec 26, 2006  │        34133 │            93 │
│  3 │ Ronald Reagan    │ Feb 6, 1911   │ June 5, 2004  │        34088 │            93 │
│  4 │ John Adams       │ Oct 30, 1735  │ July 4, 1826  │        33119 │            90 │
│  5 │ Herbert Hoover   │ Aug 10, 1874  │ Oct 20, 1964  │        32943 │            90 │
│  6 │ Harry S. Truman  │ May 8, 1884   │ Dec 26, 1972  │        32373 │            88 │
│  7 │ James Madison    │ Mar 16, 1751  │ June 28, 1836 │        31150 │      

In [69]:
#New dataframe for statistics
stats = pd.DataFrame()
#calculate mean
stats['mean'] = [df['lived_days'].mean()]
#calculate median
stats['median'] = [df['lived_days'].median()]
#calculate max value
stats['max'] = [df['lived_days'].max()]
#calculate min value
stats['min'] = [df['lived_days'].min()]
#calculate mode (using lived_years for sensible results)
stats['mode(years)'] = [df['lived_years'].mode().tolist()]

#constant weight for all values
std = df['lived_days'].std()
w_i = 1 / (std * std)
#calculate weighted average
w_avg = (df['lived_days'].sum() * w_i) / (w_i * df.shape[0])
#assign w_avg to appropriate column
stats['weighted avg.'] = [w_avg]

#print statistics table
print('Statistics (in days)')
print_table(stats.transpose(), '')

Statistics (in days)
╒═══════════════╤════════════════════╕
│ mean          │ 26423.644444444446 │
│ median        │ 26227.0            │
│ max           │ 35870              │
│ min           │ 16978              │
│ mode(years)   │ [67, 76]           │
│ weighted avg. │ 26423.644444444446 │
╘═══════════════╧════════════════════╛


In [70]:
#print statistics (in years)
stat_years = stats.copy()
sty = stats.loc[:, stats.columns != 'mode(years)'].div(365.25)
stat_years[sty.columns] = sty

#print statistics table
print('Statistics (in years)')
print_table(stat_years.transpose(), '')

Statistics (in years)
╒═══════════════╤════════════════════╕
│ mean          │ 72.343995741121    │
│ median        │ 71.80561259411363  │
│ max           │ 98.20670773442848  │
│ min           │ 46.483230663928815 │
│ mode(years)   │ [67, 76]           │
│ weighted avg. │ 72.343995741121    │
╘═══════════════╧════════════════════╛


In [71]:
#PLOTTING
#pyplot configuration
plt.style.use('default')

#copy of stats dataframe
stt = stats.copy()
#we use first value in mode array and convert to days
stt['mode'] = stt['mode(years)'][0][0] * 365.25
#drop mode(years)
stt.drop('mode(years)', axis=1, inplace=True)

In [72]:
#plot statistics (in days) as a bar graph
plot_stats(stt, 'days')

#plot statistics (in years) as a bar graph
plot_stats(stt, 'years')

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

In [73]:
#get frequencies of president ages (years)
plt.figure()
freq = df['lived_years'].value_counts()
freqmax = freq.max()
ax2 = freq.plot(kind='bar', xlabel='Age(years)', ylabel='No. of presidents')
ax2.set_yticks(range(0, freqmax + 1))
plt.savefig('plots/Age_Freq.png')
plt.close()

In [74]:
#plotting living vs dead presidents
plt.figure()
total_presidents = df.shape[0]
num_living = df['cleanedDOD'].isna().sum()
num_dead = total_presidents - num_living
ax3 = plt.bar(['Living', 'Dead'],[num_living, num_dead])
plt.xlabel('Status')
plt.ylabel('No. of presidents')
plt.savefig('plots/Status.png')
plt.close()